### Case Taxi Tip Prediction

Curso: IBM GPXX0AXIEN

Case: Predicting Taxi Tip using Scikit-Learn and Snap ML

Case criado e guiado pela IBM.

In [31]:
from __future__ import print_function
from tqdm import tqdm

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import warnings
import gc, sys

sns.set_context('notebook')

In [32]:
df = pd.read_csv('yellow_tripdata_2019-06_1M.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-06-01 0:55,2019-06-01 0:56,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0,NaN
1,1,2019-06-01 0:06,2019-06-01 0:06,1,0.0,1,N,262,263,2,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5,NaN
2,1,2019-06-01 0:17,2019-06-01 0:36,1,4.4,1,N,74,7,2,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0,NaN
3,1,2019-06-01 0:59,2019-06-01 0:59,0,0.8,1,N,145,145,2,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0,NaN
4,1,2019-06-01 0:03,2019-06-01 0:15,1,1.7,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5,NaN


##### Variável dependente (target): tip_amount

In [33]:
df.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,0.0
mean,1.634774,1.567313,3.138604,1.059064,162.520073,161.099599,1.298023,13.377135,1.122669,0.495393,2.204765,0.411648,0.298745,19.335091,2.267352,NaN
std,0.501550,1.213333,4.053755,0.693165,66.715757,70.608604,0.483148,12.486803,1.264734,0.056731,2.848908,1.709186,0.026756,15.322892,0.737953,NaN
min,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-220.000000,-5.000000,-0.500000,-11.060000,-15.000000,-0.300000,-222.800000,-2.500000,NaN
25%,1.000000,1.000000,1.000000,1.000000,114.000000,107.000000,1.000000,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,11.160000,2.500000,NaN
50%,2.000000,1.000000,1.700000,1.000000,162.000000,162.000000,1.000000,9.500000,0.500000,0.500000,1.860000,0.000000,0.300000,14.750000,2.500000,NaN
75%,2.000000,2.000000,3.210000,1.000000,233.000000,234.000000,2.000000,15.000000,2.500000,0.500000,2.950000,0.000000,0.300000,20.760000,2.500000,NaN
max,4.000000,9.000000,227.200000,99.000000,265.000000,265.000000,4.000000,579.000000,18.000000,3.300000,356.900000,88.500000,0.300000,588.920000,2.500000,NaN


In [34]:
print("Tem " + str(len(df)) + " linhas no dataset.")
print("Tem " + str(len(df.columns)) + " colunas no dataset.")

Tem 1000000 linhas no dataset.
Tem 19 colunas no dataset.


Para a var de tip_amount = 0, entendemos que a gorjeta foi paga em dinheiro, por isso, tiramos todas com 0

In [35]:
df = df[df['tip_amount'] > 0]

In [36]:
print("Tem " + str(len(df)) + " linhas no dataset.")

Tem 683507 linhas no dataset.


Tirando gorjetas maiores que o preço da corrida, porque são outliers

In [37]:
df = df[(df['tip_amount'] <= df['fare_amount'])]

In [38]:
print("Tem " + str(len(df)) + " linhas no dataset.")

Tem 682607 linhas no dataset.


Tirando viagens com o fare cost muito alto (maior que 200)

In [39]:
df = df[((df['fare_amount'] >=2) & (df['fare_amount'] < 200))]

In [40]:
print("Tem " + str(len(df)) + " linhas no dataset.")

Tem 682522 linhas no dataset.


A var total_amount contém o valor da corrida + da gorjeta, ou seja, é um ""spoiler"", por isso, dei drop

In [41]:
df.drop(['total_amount'], axis=1, inplace=True)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee
4,1,2019-06-01 0:03,2019-06-01 0:15,1,1.70,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,2.5,NaN
5,1,2019-06-01 0:28,2019-06-01 0:39,2,1.60,1,N,79,125,1,9.5,3.0,0.5,1.00,0.0,0.3,2.5,NaN
7,1,2019-06-01 0:54,2019-06-01 1:02,2,1.20,1,N,79,249,1,7.5,3.0,0.5,1.00,0.0,0.3,2.5,NaN
9,1,2019-06-01 0:29,2019-06-01 1:03,1,8.60,1,N,186,243,1,31.5,3.0,0.5,7.05,0.0,0.3,2.5,NaN
10,2,2019-06-01 0:01,2019-06-01 0:16,1,1.74,1,N,107,148,1,11.0,0.5,0.5,2.96,0.0,0.3,2.5,NaN


In [42]:
print("Tem " + str(len(df)) + " linhas no dataset.")
print("Tem " + str(len(df.columns)) + " colunas no dataset.")

Tem 682522 linhas no dataset.
Tem 18 colunas no dataset.


In [43]:
print("Tip mínima: ", np.min(df.tip_amount.values))
print("Tip máxima: ", np.max(df.tip_amount.values))
print("90% das viagens tem um tip menor ou igual a ", np.percentile(df.tip_amount.values, 90))

Tip mínima:  0.01
Tip máxima:  91.0
90% das viagens tem um tip menor ou igual a  5.88


O dataset contém informações como horários, localização, tipo de pagamento etc

Esses são valores categóricos que precisam ser tratados 

In [44]:
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])

# extraindo hora de pickup e dropoff
df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour
df['dropoff_hour'] = df['tpep_dropoff_datetime'].dt.hour

# extraindo dia da semana de pickup e dropoff
df['pickup_day'] = df['tpep_pickup_datetime'].dt.weekday
df['dropoff_day'] = df['tpep_dropoff_datetime'].dt.weekday

# tempo de viagem em minutos (float)
df['trip_time'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

In [45]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,airport_fee,pickup_hour,dropoff_hour,pickup_day,dropoff_day,trip_time
4,1,2019-06-01 00:03:00,2019-06-01 00:15:00,1,1.70,1,N,113,148,1,...,2.65,0.0,0.3,2.5,NaN,0,0,5,5,12.0
5,1,2019-06-01 00:28:00,2019-06-01 00:39:00,2,1.60,1,N,79,125,1,...,1.00,0.0,0.3,2.5,NaN,0,0,5,5,11.0
7,1,2019-06-01 00:54:00,2019-06-01 01:02:00,2,1.20,1,N,79,249,1,...,1.00,0.0,0.3,2.5,NaN,0,1,5,5,8.0
9,1,2019-06-01 00:29:00,2019-06-01 01:03:00,1,8.60,1,N,186,243,1,...,7.05,0.0,0.3,2.5,NaN,0,1,5,5,34.0
10,2,2019-06-01 00:01:00,2019-06-01 00:16:00,1,1.74,1,N,107,148,1,...,2.96,0.0,0.3,2.5,NaN,0,0,5,5,15.0


Como criamos novas colunas separadas, não precisamos mais das antigas

Além disso, a coluna de airport fee contém apenas valores NaN

In [46]:
df = df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime','airport_fee'], axis=1, inplace=False)
df.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,pickup_hour,dropoff_hour,pickup_day,dropoff_day,trip_time
4,1,1,1.70,1,N,113,148,1,9.5,3.0,0.5,2.65,0.0,0.3,2.5,0,0,5,5,12.0
5,1,2,1.60,1,N,79,125,1,9.5,3.0,0.5,1.00,0.0,0.3,2.5,0,0,5,5,11.0
7,1,2,1.20,1,N,79,249,1,7.5,3.0,0.5,1.00,0.0,0.3,2.5,0,1,5,5,8.0
9,1,1,8.60,1,N,186,243,1,31.5,3.0,0.5,7.05,0.0,0.3,2.5,0,1,5,5,34.0
10,2,1,1.74,1,N,107,148,1,11.0,0.5,0.5,2.96,0.0,0.3,2.5,0,0,5,5,15.0


As colunas categóricas precisam ser encoded (vira número). Pra isso, usamos one-hot encoding

In [47]:
get_dummy_col = ["VendorID","RatecodeID","store_and_fwd_flag","PULocationID", "DOLocationID","payment_type", "pickup_hour", "dropoff_hour", "pickup_day", "dropoff_day"]
df = pd.get_dummies(df, columns = get_dummy_col)
df.head()

,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,congestion_surcharge,trip_time,...,pickup_day_4,pickup_day_5,pickup_day_6,dropoff_day_0,dropoff_day_1,dropoff_day_2,dropoff_day_3,dropoff_day_4,dropoff_day_5,dropoff_day_6
4,1,1.70,9.5,3.0,0.5,2.65,0.0,0.3,2.5,12.0,...,False,True,False,False,False,False,False,False,True,False
5,2,1.60,9.5,3.0,0.5,1.00,0.0,0.3,2.5,11.0,...,False,True,False,False,False,False,False,False,True,False
7,2,1.20,7.5,3.0,0.5,1.00,0.0,0.3,2.5,8.0,...,False,True,False,False,False,False,False,False,True,False
9,1,8.60,31.5,3.0,0.5,7.05,0.0,0.3,2.5,34.0,...,False,True,False,False,False,False,False,False,True,False
10,1,1.74,11.0,0.5,0.5,2.96,0.0,0.3,2.5,15.0,...,False,True,False,False,False,False,False,False,True,False


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 682522 entries, 4 to 999997
Columns: 562 entries, passenger_count to dropoff_day_6
dtypes: bool(552), float64(9), int64(1)
memory usage: 416.6 MB


Transformando float64 em float32 para usar menos memória

In [49]:
df = df.astype({col: 'float32' for col in df.select_dtypes(include=['float64']).columns})

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 682522 entries, 4 to 999997
Columns: 562 entries, passenger_count to dropoff_day_6
dtypes: bool(552), float32(9), int64(1)
memory usage: 393.1 MB


In [50]:
# y com a coluna tip_amount (target)
y = df[['tip_amount']].values

# X com todas as colunas exceto tip_amount
X = df.drop(columns=['tip_amount']).values

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= 42)

In [52]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
regressor = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=6,
    max_iter=200,           
    max_leaf_nodes=31,       
    min_samples_leaf=20,     
    l2_regularization=1.0,   
    early_stopping=True,    
    validation_fraction=0.1,
    random_state=42
)
t0 = time.time()
regressor.fit(X_train, y_train)
train_sec = time.time() - t0
print(f"Tempo de treino: {train_sec:.2f}s")

Tempo de treino: 26.64s


In [54]:
print("R^2 (teste):", regressor.score(X_test, y_test))

R^2 (teste): 0.782495151748214


In [55]:
y_pred = regressor.predict(X_test)

In [56]:
# média da diferença absoluta entre o valor real e o valor previsto
# quanto menor, melhor
mae = mean_absolute_error(y_test, y_pred)

# média dos erros elevados ao quadrado
# erros grandes ficam muito maiores
# quanto menor, melhor
mse = mean_squared_error(y_test, y_pred)

# raiz quadrada do MSE, voltando à escala original do seu alvo
# quanto menor, melhor
rmse = np.sqrt(mse)

# mede quanto da variabilidade do alvo o modelo explica
# quanto maior, melhor (vai até 1)
r2 = r2_score(y_test, y_pred)

print(f"MAE:  {mae:.4f}")
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")

MAE:  0.7026
MSE:  1.5533
RMSE: 1.2463
R²:   0.7825
